In [ ]:
import numpy as np
import pandas as pd
import subprocess
import h5py
import sys
from numba import njit
import matplotlib.pyplot as plt
from typing import List, Dict
import os
import glob
plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = (12,10)
import json
import time
import matplotlib
from pathlib import Path
from scipy.interpolate import CubicSpline
spec_home="/home/himanshu/spec/my_spec"
matplotlib.matplotlib_fname()

# Various functions to read across levs
### Also functions to make reports

In [ ]:
def read_dat_file(file_name):
  cols_names = []

  temp_file = "./temp.csv"
  with open(file_name,'r') as f:
    with open(temp_file,'w') as w:
      lines = f.readlines()
      for line in lines:
        if(line[0] != '#'): # This is data
          w.writelines(line)
        if(line[0:3] == '# [' or line[0:4] == '#  ['): # Some dat files have comments on the top
          cols_names.append(line.split('=')[-1][1:-1].strip())


  return pd.read_csv(temp_file,delim_whitespace=True,names=cols_names)

# Files like AhACoefs.dat have unequal number of columns
def read_dat_file_uneq_cols(file_name):
  cols_names = []

  temp_file = "./temp.csv"
  col_length = 0
  with open(file_name,'r') as f:
    with open(temp_file,'w') as w:
      lines = f.readlines()
      for line in lines:
        if(line[0] != '#'): # This is data
          w.writelines(" ".join(line.split()[:col_length])+"\n")
        if(line[0:3] == '# [' or line[0:4] == '#  ['): # Some dat files have comments on the top
          cols_names.append(line.split('=')[-1][1:-1].strip())
          col_length = col_length+1


  return pd.read_csv(temp_file,delim_whitespace=True,names=cols_names)

def plot_and_save(data,x_arr,y_arr,save_folder,file_name):
  for x_axis,y_axis in zip(x_arr,y_arr):
    plt.plot(data[x_axis],data[y_axis])
    plt.xlabel(x_axis)
    plt.ylabel(y_axis)
    title = file_name[:-4]+" \""+y_axis+"\" vs \""+x_axis+"\""
    plt.title(title)
    save_path = save_folder+title.replace("/","_")
    print(f"Plotted: {save_path}" )
    print(os.getcwd())

    plt.savefig(save_path)
    plt.clf()



def read_dat_file_across_AA(file_pattern):

  path_pattern = file_pattern
  path_collection = []


  for folder_name in glob.iglob(path_pattern, recursive=True):
      if os.path.isdir(folder_name) or os.path.isfile(folder_name):
          path_collection.append(folder_name)
          print(folder_name)


  read_data_collection = []
  for path in path_collection:
    # AhACoefs.dat has uneq cols
    if "Coefs.dat" in path:
        read_data_collection.append(read_dat_file_uneq_cols(path))
    else:
        read_data_collection.append(read_dat_file(path))

  data = pd.concat(read_data_collection)
  # print(data.columns)
  return data

def read_AH_files(Ev_path):
  fileA = Ev_path + "Run/ApparentHorizons/AhA.dat"
  fileB = Ev_path + "Run/ApparentHorizons/AhB.dat"

  dataA = read_dat_file_across_AA(fileA)
  dataB = read_dat_file_across_AA(fileB)

  return dataA,dataB  

  
# Combines all the pvd files into a single file and save it in the base folder
def combine_pvd_files(base_folder:Path, file_pattern:str, output_path=None):
  pvd_start ="""<?xml version="1.0"?>\n<VTKFile type="Collection" version="0.1" byte_order="LittleEndian">\n  <Collection>\n"""
  pvd_end ="  </Collection>\n</VTKFile>"

  vis_folder_name = file_pattern.split("/")[-1][:-4]
  Lev = file_pattern[0:4]

  if output_path is None:
    output_path = f"{base_folder}/{vis_folder_name}_{Lev}.pvd"

  pvd_files = list(base_folder.glob(file_pattern))
  pvd_folders = list(base_folder.glob(file_pattern[:-4]))


  with open(output_path,'w') as write_file:
    write_file.writelines(pvd_start)
    for files in pvd_files:
      print(files)
      with files.open("r") as f:
        for line in f.readlines():
          line = line.replace(vis_folder_name,str(files)[:-4])
          if "DataSet" in line:
            write_file.writelines(line)
    write_file.writelines(pvd_end)
  
  print(output_path)

def moving_average(array,avg_len):
    return np.convolve(array,np.ones(avg_len))/avg_len
    
def moving_average_valid(array,avg_len):
    return np.convolve(array,np.ones(avg_len),'valid')/avg_len


def path_to_folder_name(folder_name):
  return folder_name.replace("/","_")

# Give a dict of {"run_name" = runs_path} and data_file_path to get {"run_name" = dat_file_data}
def load_data_from_levs(runs_path, data_file_path):
  data_dict = {}
  column_list = ""
  for run_name in runs_path.keys():
    data_dict[run_name] = read_dat_file_across_AA(runs_path[run_name]+data_file_path)
    column_list = data_dict[run_name].columns
  return column_list, data_dict


def plot_graph_for_runs(runs_data_dict, x_axis, y_axis, minT, maxT, legend_dict=None, save_path=None, moving_avg_len=0, plot_fun = lambda x,y,label : plt.plot(x,y,label=label),sort_by=None):
  sort_run_data_dict(runs_data_dict,sort_by=sort_by)
  minT_indx_list={}
  maxT_indx_list={}

  if legend_dict is None:
    legend_dict = {}
    for run_name in runs_data_dict.keys():
      legend_dict[run_name] = None
  else:
    for run_name in runs_data_dict.keys():
      if run_name not in legend_dict:
        raise ValueError(f"{run_name} not in {legend_dict=}")

  
  for run_name in runs_data_dict.keys():
    minT_indx_list[run_name] = len(runs_data_dict[run_name][x_axis][runs_data_dict[run_name][x_axis] < minT])
    maxT_indx_list[run_name] = len(runs_data_dict[run_name][x_axis][runs_data_dict[run_name][x_axis] < maxT])

  if moving_avg_len == 0:

    for run_name in runs_data_dict.keys():
      x_data = runs_data_dict[run_name][x_axis][minT_indx_list[run_name]:maxT_indx_list[run_name]]
      y_data = runs_data_dict[run_name][y_axis][minT_indx_list[run_name]:maxT_indx_list[run_name]]
      

    #   print(f"{len(x_data)=},{len(y_data)=},{len(np.argsort(x_data))=},{type(x_data)=}")

    #   sorted_indices = x_data.argsort()
    #   x_data = x_data.iloc[sorted_indices]
    #   y_data = y_data.iloc[sorted_indices]
      legend = legend_dict[run_name]
      if legend is None:
        legend = run_name
      plot_fun(x_data, y_data,legend)

    plt.xlabel(x_axis)
    plt.ylabel(y_axis)
    title = "\"" +  y_axis+"\" vs \""+x_axis+"\""
    plt.title(title)
    plt.legend()

  else:
    for run_name in runs_data_dict.keys():
      x_data = np.array(runs_data_dict[run_name][x_axis][minT_indx_list[run_name] + moving_avg_len-1:maxT_indx_list[run_name]])
      y_data = np.array(moving_average_valid(runs_data_dict[run_name][y_axis][minT_indx_list[run_name]:maxT_indx_list[run_name]], moving_avg_len))

    #   sorted_indices = np.argsort(x_data)
    #   x_data = x_data[sorted_indices]
    #   y_data = y_data[sorted_indices]
      legend = legend_dict[run_name]
      if legend is None:
        legend = run_name
      plot_fun(x_data, y_data,legend)

    plt.xlabel(x_axis)
    plt.ylabel(y_axis)
    title = "\"" + y_axis+ "\" vs \"" + x_axis + "\"  " + f"avg_window_len={moving_avg_len}"
    plt.title(title)
    plt.legend()

  
  if save_path is not None:
    fig_x_label = x_axis.replace("/","_").replace(".","_")
    fig_y_label = y_axis.replace("/","_").replace(".","_")
    save_file_name = f"{fig_y_label}_vs_{fig_x_label}_minT={minT}_maxT={maxT}_moving_avg_len={moving_avg_len}"
    for run_name in runs_data_dict.keys():
      save_file_name = save_file_name + "__" + run_name

    plt.savefig(save_path+save_file_name)


def find_file(pattern):
  return glob.glob(pattern, recursive=True)[0]

def plots_for_a_folder(things_to_plot,plot_folder_path,data_folder_path):
  for plot_info in things_to_plot:
    file_name = plot_info['file_name']
    y_arr = plot_info['columns'][1:]
    x_arr = [plot_info['columns'][0]]*len(y_arr)

    data = read_dat_file_across_AA(data_folder_path+"/**/"+file_name)
    plot_and_save(data,x_arr,y_arr,plot_folder_path,file_name)

def is_the_current_run_going_on(run_folder):
  if len(find_file(run_folder+"/**/"+"TerminationReason.txt")) > 0:
    return False
  else:
    return True

def plot_min_grid_spacing(runs_data_dict):
    '''
    runs_data_dict should have dataframes with MinimumGridSpacing.dat data.
    The function will compute the min grid spacing along all domains and plot it.
    '''
    keys = runs_data_dict.keys()
    if len(keys) == 0:
        print("There are no dataframes in the dict")

    for key in keys:
        t_step = runs_data_dict[key]['t']
        min_val = runs_data_dict[key].drop(columns=['t']).min(axis='columns')
        plt.plot(t_step,min_val,label=key)

    plt.legend()
    plt.xlabel("t")
    plt.ylabel("Min Grid Spacing")
    plt.title("Min grid spacing in all domains")
    plt.show()

def plot_GrAdjustSubChunksToDampingTimes(runs_data_dict):
    keys = runs_data_dict.keys()
    if len(keys) > 1:
        print("To plot the Tdamp for various quantities only put one dataframe in the runs_data_dict")

    data:pd.DataFrame = runs_data_dict[list(keys)[0]]
    tdamp_keys = []
    for key in data.keys():
        if 'Tdamp' in key:
            tdamp_keys.append(key)

    # Get a colormap
    cmap = plt.get_cmap('tab10')
    colors = cmap(np.linspace(0, 1, len(tdamp_keys)))

    t_vals = data['time']
    for i, color, key in zip(range(len(tdamp_keys)),colors, tdamp_keys):
        if i%2==0:
            plt.plot(t_vals,data[key],label=key,color=color)
        else:
            plt.plot(t_vals,data[key],label=key,color=color,linestyle="--")


    min_tdamp = data[tdamp_keys].min(axis='columns')
    plt.plot(t_vals,min_tdamp,label="min_tdamp",linewidth=3,linestyle="dotted",color="red")

    plt.legend()
    plt.xlabel("time")
    plt.title(list(keys)[0])
    plt.show()

def add_max_and_min_val(runs_data_dict):
    # If we load a file with 5 columns with first being time, then find max and min values for all the other columns, at all times and add it to the dataframe.
    # Useful when you want to find like Linf across all domains at all times
    for run_name in runs_data_dict.keys():
        data_frame = runs_data_dict[run_name]
        t = data_frame.iloc[:,0]
        max_val = np.zeros_like(t)
        min_val = np.zeros_like(t)
        for i in range(len(t)):
            max_val[i] = data_frame.iloc[i,1:].max()
            min_val[i] = data_frame.iloc[i,1:].max()

        # Add the values to the dataframe
        data_frame['max_val'] = max_val
        data_frame['min_val'] = min_val

def sort_run_data_dict(runs_data_dict:dict,sort_by=None):
    for run_name in runs_data_dict.keys():
        run_df = runs_data_dict[run_name]
        if sort_by is None:
            sort_by = run_df.keys()[0]
        runs_data_dict[run_name] = run_df.sort_values(by=sort_by)

## Functions to read horizon files

In [ ]:
def make_Bh_pandas(h5_dir):
    # Empty dataframe
    df = pd.DataFrame()
    
    # List of all the vars in the h5 file
    var_list = []
    h5_dir.visit(var_list.append)
    
    
    for var in var_list:
        # This means there is no time column
        # print(f"{var} : {h5_dir[var].shape}")
        if df.shape == (0,0):
            # data[:,0] is time and then we have the data
            data = h5_dir[var]
            
            # vars[:-4] to remove the .dat at the end
            col_names = make_col_names(var[:-4],data.shape[1]-1)
            col_names.append('t')
            # Reverse the list so that we get ["t","var_name"]
            col_names.reverse()            
            append_to_df(data[:],col_names,df)
            
        else:
            data = h5_dir[var]
            col_names = make_col_names(var[:-4],data.shape[1]-1)         
            append_to_df(data[:,1:],col_names,df)
            
    return df

def append_to_df(data,col_names,df):
    for i,col_name in enumerate(col_names):
        df[col_name] = data[:,i]
        
def make_col_names(val_name:str,val_size:int):
    col_names = []
    if val_size == 1:
        col_names.append(val_name)
    else:
        for i in range(val_size):
            col_names.append(val_name+f"_{i}")
    return col_names


def horizon_to_pandas(horizon_path:Path):
    assert(horizon_path.exists())
    df_dict = {}
    with h5py.File(horizon_path,'r') as hf:
        # Not all horizon files may have AhC
        for key in hf.keys():
            if key == 'VersionHist.ver':
                # Newer runs have this
                continue
            df_dict[key[:-4]] = make_Bh_pandas(hf[key])

    return df_dict

def read_horizon_across_Levs(path_list:List[Path]):
    df_listAB = []
    df_listC = []
    final_dict = {}
    for path in path_list:
        df_lev = horizon_to_pandas(path)
        # Either [AhA,AhB] or [AhA,AhB,AhC]
        if len(df_lev.keys()) > 1:
            df_listAB.append(df_lev)
        # Either [AhC] or [AhA,AhB,AhC]
        if (len(df_lev.keys()) == 1) or (len(df_lev.keys()) ==3):
            df_listC.append(df_lev)
    if len(df_listAB)==1:
        # There was only one lev
        final_dict = df_listAB[0]
    else:
        final_dict["AhA"] = pd.concat([df["AhA"] for df in df_listAB])
        final_dict["AhB"] = pd.concat([df["AhB"] for df in df_listAB])
        if len(df_listC) > 0:
            final_dict["AhC"] = pd.concat([df["AhC"] for df in df_listC])       
    
    return final_dict

def load_horizon_data_from_levs(base_path:Path, runs_path:Dict[str,Path]):
  data_dict = {}
  for run_name in runs_path.keys():
    path_list = list(base_path.glob(runs_path[run_name]))
    print(path_list)
    data_dict[run_name] = read_horizon_across_Levs(path_list)
  return data_dict

def flatten_dict(horizon_data_dict:Dict[str,pd.DataFrame]) -> Dict[str,pd.DataFrame] :
  flattened_data = {}
  for run_name in horizon_data_dict.keys():
      for horizons in horizon_data_dict[run_name]:
          flattened_data[run_name+"_"+horizons] = horizon_data_dict[run_name][horizons]
          # print(run_name+"_"+horizons)
  return flattened_data

# Plot dat files

In [ ]:
runs_to_plot = {}
# runs_to_plot["boost_ID_test_wrong"] =  "/groups/sxs/hchaudha/spec_runs/boost_ID_test_wrong/Ev/Lev3_A?/Run/"
# runs_to_plot["boost_ID_test_correct"] =  "/groups/sxs/hchaudha/spec_runs/boost_ID_test_correct/Ev/Lev3_A?/Run/"
# runs_to_plot["corrected_coord_spin1"] =  "/groups/sxs/hchaudha/spec_runs/corrected_coord_spin1/Ev/Lev3_A?/Run/"
# runs_to_plot["corrected_coord_spin2"] =  "/groups/sxs/hchaudha/spec_runs/corrected_coord_spin2/Ev/Lev3_A?/Run/"
# runs_to_plot["2_SpKS_q1_sA_0_0_0_sB_0_0_0_d15"] =  "/groups/sxs/hchaudha/spec_runs/2_SpKS_q1_sA_0_0_0_sB_0_0_0_d15/Ev/Lev3_A?/Run/"
# runs_to_plot["2_SpKS_q1_sA_0_0_0_sB_0_0_99_d15"] =  "/groups/sxs/hchaudha/spec_runs/2_SpKS_q1_sA_0_0_0_sB_0_0_99_d15/Ev/Lev3_A?/Run/"
# runs_to_plot["2_SpKS_q1_sA_0_0_0_sB_0_0_9_d15"] =  "/groups/sxs/hchaudha/spec_runs/2_SpKS_q1_sA_0_0_0_sB_0_0_9_d15/Ev/Lev3_A?/Run/"
# runs_to_plot["2_SpKS_q1_sA_0_0_9_sB_0_0_9_d15"] =  "/groups/sxs/hchaudha/spec_runs/2_SpKS_q1_sA_0_0_9_sB_0_0_9_d15/Ev/Lev3_A?/Run/"
runs_to_plot["3_DH_q1_ns_d18_L3"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3/Ev/Lev3_A?/Run/"
runs_to_plot["3_DH_q1_ns_d18_L6"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L6/Ev/Lev6_A?/Run/"
runs_to_plot["all_10"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_10/Lev3_A?/Run/"
runs_to_plot["all_100"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100/Lev3_A?/Run/"
runs_to_plot["near_bhs_10"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/near_bhs_10/Lev3_A?/Run/"
runs_to_plot["near_bhs_100"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/near_bhs_100/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_ns_d18_L6_AA"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L6_AA/Ev/Lev3_A?/Run/"
# runs_to_plot["4_SphKS_q1_15_SSphKS_ID"] =  "/groups/sxs/hchaudha/spec_runs/4_SphKS_q1_15_SSphKS_ID/Ev/Lev3_A?/Run/"
# runs_to_plot["4_SphKS_q1_15_SKS_ID"] =  "/groups/sxs/hchaudha/spec_runs/4_SphKS_q1_15_SKS_ID/Ev/Lev3_A?/Run/"
# runs_to_plot["5_gd_SphKS_gauge_ID"] =  "/groups/sxs/hchaudha/spec_runs/5_gd_SphKS_gauge_ID/Ev/Lev2_A?/Run/"
# runs_to_plot["5_ngd_SphKS_ID"] =  "/groups/sxs/hchaudha/spec_runs/5_ngd_SphKS_ID/Ev/Lev2_A?/Run/"
# runs_to_plot["5_ngd_KS_ID"] =  "/groups/sxs/hchaudha/spec_runs/5_ngd_KS_ID/Ev/Lev2_A?/Run/"



# data_file_path="ConstraintNorms/GhCe_Norms.dat"
data_file_path="ConstraintNorms/GhCe.dat"
# data_file_path="ConstraintNorms/GhCe_Linf.dat"
# data_file_path="ConstraintNorms/GhCe_L2.dat"
# data_file_path="ConstraintNorms/GhCe_VolL2.dat"
# data_file_path="ConstraintNorms/NormalizedGhCe_Norms.dat"
# data_file_path="ConstraintNorms/NormalizedGhCe_Linf.dat"
# data_file_path="CharSpeedNorms/CharSpeeds_Min_SliceLFF.SphereA0.dat"
# data_file_path="MinimumGridSpacing.dat"
# data_file_path="GrAdjustMaxTstepToDampingTimes.dat"
# data_file_path="GrAdjustSubChunksToDampingTimes.dat"
# data_file_path="DiagAhSpeedA.dat"
# data_file_path="ApparentHorizons/AhA.dat"
# data_file_path="ApparentHorizons/AhB.dat"
# data_file_path="ApparentHorizons/MinCharSpeedAhA.dat"
# data_file_path="ApparentHorizons/RescaledRadAhA.dat"
# data_file_path="ApparentHorizons/AhACoefs.dat"
# data_file_path="ApparentHorizons/Trajectory_AhB.dat"
# data_file_path="ApparentHorizons/HorizonSepMeasures.dat"
# data_file_path="TStepperDiag.dat"
# data_file_path="TimeInfo.dat"


column_names, runs_data_dict = load_data_from_levs(runs_to_plot,data_file_path)
print(column_names)

In [ ]:
moving_avg_len=0
save_path = None
# add_max_and_min_val(runs_data_dict)
# y_axis = 'max_val'
# y_axis = 'min_val'
x_axis = 'time'
# y_axis = 'Shape_NumberOfPiledUpModes'
y_axis = 'Linf(GhCe)'
y_axis = 'L2(GhCe)'
y_axis = 'VolLp(GhCe)' 
# y_axis = 'VolLp(sqrt(1Con^2))'
# y_axis = 'VolLp(GhCe) on SphereA0'
# y_axis = 'Linf(GhCe) on SphereB0'
# y_axis = 'Linf(GhCe) on SphereA0'
# y_axis = 'Linf(GhCe) on CylinderSMB0.0'
# y_axis = 'U0'
# y_axis = 'VolLp(sqrt(1Con^2))'
# y_axis = 'VolLp(sqrt(2Con^2))'
# y_axis = 'Linf(NormalizedGhCe)'
# y_axis = 'MaxAllowedTstep'
# y_axis = 'Tdamp(SmoothMinDeltaRNoLam00AhB)'
# y_axis = 'Tdamp(LambdaFactorA0)'

# x_axis = 't'
# y_axis = 'T [hours]'
# y_axis = 'dt/dT'

# x_axis = 't'
# y_axis = 'CoordSepHorizons'
# y_axis = 'ProperSepHorizons'
# y_axis = 'MinCharSpeedAhA[1]'
# y_axis = 'MinimumGridSpacing[SphereB0]'
# y_axis = 'MinimumGridSpacing[CylinderSMB0.0]'
# y_axis = 'dt/dT'


# x_axis = 'time'
# y_axis = 'L_surface'
# y_axis = 'Area'
# y_axis = 'NumIterations'
# y_axis = 'max(R_ij)'
# y_axis = 'Coef(1,-1)'
# y_axis = 'Shape_NumberOfPiledUpModes'
# y_axis = 'max(r)'
# y_axis = 'min(r)'
# y_axis = 'max(|r^i-c^i|)'


# y_axis = 'courant factor'
# y_axis = 'error/1e-08'
# y_axis = 'NfailedSteps'
# y_axis = 'NumRhsEvaluations in this segment'

# x_axis = 'time after step'

# y_axis = 'dt'
# moving_avg_len=50

minT = 0
# minT = 2000
maxT = 40000
# maxT = 4000

plot_fun = lambda x,y,label : plt.plot(x,y,label=label)
plot_fun = lambda x,y,label : plt.semilogy(x,y,label=label)
# plot_fun = lambda x,y,label : plt.loglog(x,y,label=label) 
# plot_fun = lambda x,y,label : plt.scatter(x,y,label=label,s=10,marker="x")
# save_path = "/groups/sxs/hchaudha/rough/high_acc_plots/"
# save_path = "/groups/sxs/hchaudha/rough/plots/"
legend_dict = {}
for key in runs_data_dict.keys():
  legend_dict[key] = None

# legend_dict = { '3_DH_q1_ns_d18_L3': "Lev3",
#                 '3_DH_q1_ns_d18_L6': "Lev6",
#                 'all_10': "Lev3_all_tols_10",
#                 'all_100': "Lev3_all_tols_100",
#                 'near_bhs_10': "Lev3_sphere_AB_tols_10",
#                 'near_bhs_100': "Lev3_sphere_AB_tols_100"}

with plt.style.context('default'):
  plt.rcParams["figure.figsize"] = (12,10)
  plt.rcParams["figure.autolayout"] = True
  plot_graph_for_runs(runs_data_dict, x_axis, y_axis, minT, maxT, legend_dict=legend_dict, save_path=save_path, moving_avg_len=moving_avg_len, plot_fun=plot_fun)
  plt.show()

In [ ]:
runs_data_dict.keys()

In [ ]:
data_file_path="ConstraintNorms/GhCe_L2.dat"
data_file_path="ConstraintNorms/GhCe_Linf.dat"
data_file_path="ConstraintNorms/GhCe_VolL2.dat"
column_names, runs_data_dict = load_data_from_levs(runs_to_plot,data_file_path)
# print(column_names)
print(runs_data_dict.keys())

In [ ]:
# run_name = list(runs_data_dict.keys())[4]
run_name = 'near_bhs_100'
df = runs_data_dict[run_name].copy()
df = df.sort_values(by=df.columns[0])

tmin=2050
# tmin=0
tmax=4000
# tmax=50000

df = df[df['time']>=tmin]
df = df[df['time']<tmax]
t_name = df.columns[0]
y_axis = df.columns[1].split(" ")[0]
all_cols_but_t = df.columns[1:]
# Find the maximum value across columns B, C, D, and F for each row
df['max_val'] = df[all_cols_but_t].max(axis=1)

# Determine which column had the maximum value
df['max_source'] = df[all_cols_but_t].idxmax(axis=1)

# List all columns that have at least one max value
columns_with_max = df['max_source'].unique()

# Generate a colormap for the columns with at least one max value
num_colors = len(columns_with_max)
colors = plt.get_cmap('tab20', num_colors)  # Using 'tab20' colormap
color_map = {column: colors(i) for i, column in enumerate(columns_with_max)}

# Plot max_BCD vs t with different colors for different sources
plt.figure(figsize=(18, 10))
for i,source in enumerate(columns_with_max):
    subset = df[df['max_source'] == source]
    if i%4 == 0:
        plt.scatter(subset[t_name], subset['max_val'], color=color_map[source], label=source, s=10, marker="^")
    if i%4 == 1:
        plt.scatter(subset[t_name], subset['max_val'], color=color_map[source], label=source, s=10, marker="v")
    if i%4 == 2:
        plt.scatter(subset[t_name], subset['max_val'], color=color_map[source], label=source, s=10, marker=">")
    if i%4 == 3:
        plt.scatter(subset[t_name], subset['max_val'], color=color_map[source], label=source, s=10, marker="<")

plt.xlabel(t_name)
plt.ylabel(y_axis)
plt.yscale('log')
plt.title(f'{y_axis} vs {t_name} for {run_name}')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(f"/groups/sxs/hchaudha/rough/{run_name}_{y_axis}_vs_{t_name}_{tmin}_{tmax}.png", dpi=500)
plt.show()

In [ ]:
data_file_path="MinimumGridSpacing.dat"
column_names, runs_data_dict = load_data_from_levs(runs_to_plot,data_file_path)
sort_run_data_dict(runs_data_dict)
plot_min_grid_spacing(runs_data_dict)

In [ ]:
data_file_path="GrAdjustSubChunksToDampingTimes.dat"
column_names, runs_data_dict = load_data_from_levs(runs_to_plot,data_file_path)
sort_run_data_dict(runs_data_dict)

In [ ]:
print(runs_data_dict.keys())
idx = 0
runs_data_dict_temp = {f"{list(runs_data_dict.keys())[idx]}":runs_data_dict[list(runs_data_dict.keys())[idx]]}
plot_GrAdjustSubChunksToDampingTimes(runs_data_dict_temp)

# Plot horizons.h5

In [ ]:
current_dir = os.getcwd()
if "hchaudha" in current_dir:
    base_path = Path("/groups/sxs/hchaudha/spec_runs")
else:
    base_path = Path("/panfs/ds09/sxs/himanshu/gauge_stuff/gauge_driver_runs/runs")

In [ ]:
runs_to_plot = {}
# runs_to_plot["76_ngd_master_mr1_50_3000"] =  "76_ngd_master_mr1_50_3000/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["76_ngd_master_mr1_200_3000"] =  "76_ngd_master_mr1_200_3000/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["77_gd_Kerr_q1"] =  "77_gd_Kerr_q1/Ev/Lev1_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["77_gd_Kerr_q3"] =  "77_gd_Kerr_q3/Ev/Lev1_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["77_gd_Kerr_q1_Kerr"] =  "77_gd_Kerr_q1/Ev_Kerr/Lev1_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["77_gd_Kerr_q3_Kerr"] =  "77_gd_Kerr_q3/Ev_Kerr/Lev1_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["78_ngd_master_mr1"] =  "78_ngd_master_mr1/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["79_ngd_master_mr1_1000_3000"] =  "79_ngd_master_mr1_1000_3000/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["79_ngd_master_mr1_1000_3000_high_tol"] =  "79_ngd_master_mr1_1000_3000/Ev_AH_high_tol/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["79_ngd_master_mr1_200_3000"] =  "79_ngd_master_mr1_200_3000/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["80_ngd_master_mr1_100"] =  "80_ngd_master_mr1_100/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["80_ngd_master_mr1_50"] =  "80_ngd_master_mr1_50/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["80_ngd_master_mr1_10"] =  "80_ngd_master_mr1_10/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["80_ngd_master_mr1_5"] =  "80_ngd_master_mr1_5/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["80_ngd_master_mr1_300"] =  "80_ngd_master_mr1_300/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["81_gd_Kerr_q3_0_9_0__0_0_0"] =  "81_gd_Kerr_q3_0_9_0__0_0_0/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["81_gd_DH_q3_0_9_0__0_0_0"] =  "81_gd_DH_q3_0_9_0__0_0_0/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["82_ngd_master_mr1_50_3000_DH_to_DH"] =  "82_ngd_master_mr1_50_3000_DH_to_DH/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["83_ngd_master_mr1_200_3000_no_eps"] =  "83_ngd_master_mr1_200_3000_no_eps/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["83_ngd_master_mr1_200_3000_no_eps_high_tol"] =  "83_ngd_master_mr1_200_3000_no_eps/Ev_AH_high_tol/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["83_ngd_master_mr1_200_3000_no_eps_high_tol_all"] =  "83_ngd_master_mr1_200_3000_no_eps/Ev_high_tol_all/Lev3_A?/Run/"
# runs_to_plot["83_ngd_master_mr1_200_3000_no_eps_high_tol_all_100"] =  "83_ngd_master_mr1_200_3000_no_eps/Ev_high_tol_all_100/Lev3_A?/Run/"
# runs_to_plot["83_ngd_master_mr1_200_3000_no_eps_high_tol"] =  "83_ngd_master_mr1_200_3000_no_eps/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["83_ngd_master_mr1_200_3000_no_eps_no_lsr"] =  "83_ngd_master_mr1_200_3000_no_eps_no_lsr/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["84_gd_KerrI_3000_200"] =  "84_gd_KerrI_3000_200/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["84_gd_DH_3000_200"] =  "84_gd_DH_3000_200/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["83_no_eps_Ev_wrong_evolution"] =  "83_ngd_master_mr1_200_3000_no_eps/Ev_wrong_evolution/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["83_no_eps_Ev_pow2"] =  "83_ngd_master_mr1_200_3000_no_eps/Ev_pow2/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["83_no_eps_Ev_pow6"] =  "83_ngd_master_mr1_200_3000_no_eps/Ev_pow6/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["83_no_eps_Ev_tanh15"] =  "83_ngd_master_mr1_200_3000_no_eps/Ev_tanh15/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["83_no_eps_Ev_tanh7"] =  "83_ngd_master_mr1_200_3000_no_eps/Ev_tanh7/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["83_no_eps_Ev_tanh7_lsr_correct_evolution"] =  "83_ngd_master_mr1_200_3000_no_eps/Ev_tanh7_lsr_correct_evolution/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["110_KerrToDHTanh_1"] =  "110_KerrToDHTanh_1/Ev/Lev1_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["110_KerrToDHTanh_3"] =  "110_KerrToDHTanh_3/Ev/Lev1_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["113_KerrToDHTanh_st1_scP01"] =  "113_KerrToDHTanh_st1_scP01/Ev/Lev1_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["113_KerrToDHTanh_st1_scP05"] =  "113_KerrToDHTanh_st1_scP05/Ev/Lev1_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["113_KerrToDHTanh_st1_scP1"] =  "113_KerrToDHTanh_st1_scP1/Ev/Lev1_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["113_KerrToDHTanh_st1_scP5"] =  "113_KerrToDHTanh_st1_scP5/Ev/Lev1_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["ID_SUKS2_1_0,0,0_0,0,0_10_gd_main"] =  "ID_SUKS2_1_0,0,0_0,0,0_10_gd_main/Ev/Lev1_A?/Run/ApparentHorizons/Horizons.h5"

# runs_to_plot["67_master_mr1"] =  "67_master_mr1/Ev/Lev1_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["master_mr1_Lev1"] =  "master_mr1/Ev/Lev1_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["master_mr1_Lev3"] =  "master_mr1/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["AccTest_q1ns_Lev3"] =  "AccTest_q1ns_Lev3/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["AccTest_q1ns_Lev5"] =  "AccTest_q1ns_Lev5/Ev/Lev5_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["AccTest_q1ns_Lev7"] =  "AccTest_q1ns_Lev7/Ev/Lev7_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["AccTest_q1ns_Lev9"] =  "AccTest_q1ns_Lev9/Ev/Lev9_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["AccTest_q1ns_Lev6p"] =  "AccTest_q1ns_Lev5/Ev_Lev6/Lev5_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["AccTest_q1ns_Lev7p"] =  "AccTest_q1ns_Lev5/Ev_Lev7/Lev5_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["AccTest_q1ns_Lev8p"] =  "AccTest_q1ns_Lev5/Ev_Lev8/Lev5_A?/Run/ApparentHorizons/Horizons.h5"

# runs_to_plot["HQ_3_gd_SUKS_L5_5_15"] =  "HQ_3_gd_SUKS_L5_5_15/Ev/Lev5_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["HQ_3_gd_SKS_L5_5_15"] =  "HQ_3_gd_SKS_L5_5_15/Ev/Lev5_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["HQ_3_ngd_DH_L5_5_15"] =  "HQ_3_ngd_DH_L5_5_15/Ev/Lev5_A?/Run/ApparentHorizons/Horizons.h5"

runs_to_plot["2_SpKS_q1_sA_0_0_0_sB_0_0_0_d15"] =  "2_SpKS_q1_sA_0_0_0_sB_0_0_0_d15/Ev/Lev3_AA/Run/ApparentHorizons/Horizons.h5"
runs_to_plot["2_SpKS_q1_sA_0_0_0_sB_0_0_99_d15"] =  "2_SpKS_q1_sA_0_0_0_sB_0_0_99_d15/Ev/Lev3_AA/Run/ApparentHorizons/Horizons.h5"
runs_to_plot["2_SpKS_q1_sA_0_0_0_sB_0_0_9_d15"] =  "2_SpKS_q1_sA_0_0_0_sB_0_0_9_d15/Ev/Lev3_AA/Run/ApparentHorizons/Horizons.h5"
runs_to_plot["2_SpKS_q1_sA_0_0_9_sB_0_0_9_d15"] =  "2_SpKS_q1_sA_0_0_9_sB_0_0_9_d15/Ev/Lev3_AA/Run/ApparentHorizons/Horizons.h5"

# runs_to_plot["corrected_coord_spin2"] =  "corrected_coord_spin2/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["boost_ID_test_wrong"] =  "boost_ID_test_wrong/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["boost_ID_test_correct"] = "boost_ID_test_correct/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["corrected_coord_spin1"] =  "corrected_coord_spin1/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"

data_dict = load_horizon_data_from_levs(base_path, runs_to_plot)
data_dict = flatten_dict(data_dict)
data_dict[list(data_dict.keys())[0]].columns

In [ ]:
moving_avg_len = 0
save_path = None

# x_axis = 't'
# y_axis = 'ArealMass'
y_axis = 'ChristodoulouMass'
# y_axis = 'CoordCenterInertial_0'
# y_axis = 'CoordCenterInertial_1'
# y_axis = 'CoordCenterInertial_2'
# y_axis = 'DimensionfulInertialSpin_0'
# y_axis = 'DimensionfulInertialSpin_1'
# y_axis = 'DimensionfulInertialSpin_2'
# y_axis = 'DimensionfulInertialCoordSpin_0'
# y_axis = 'DimensionfulInertialCoordSpin_1'
# y_axis = 'DimensionfulInertialCoordSpin_2'
# y_axis = 'DimensionfulInertialSpinMag'
# y_axis = 'SpinFromShape_0'
# y_axis = 'SpinFromShape_1'
# y_axis = 'SpinFromShape_2'
# y_axis = 'SpinFromShape_3'
# y_axis = 'chiInertial_0'
# y_axis = 'chiInertial_1'
# y_axis = 'chiInertial_2'
# y_axis = 'chiMagInertial'



# moving_avg_len=25
minT = 0
maxT = 800

plot_fun = lambda x,y,label : plt.plot(x,y,label=label)
# plot_fun = lambda x,y,label : plt.semilogy(x,y,label=label)
# plot_fun = lambda x,y,label : plt.loglog(x,y,label=label)
# plot_fun = lambda x,y,label : plt.scatter(x,y,label=label)
# save_path = "/panfs/ds09/sxs/himanshu/scripts/report/not_tracked/temp2/"

filtered_dict = {}
allowed_horizons = ["AhB"]
for horizons in allowed_horizons:
  for runs_keys in data_dict.keys():
    if horizons in runs_keys:
      filtered_dict[runs_keys] = data_dict[runs_keys]
 
with plt.style.context('default'):
  plt.rcParams["figure.figsize"] = (12,10)
  plt.rcParams["figure.figsize"] = (8,6)
  plt.rcParams["figure.autolayout"] = True
  plot_graph_for_runs(filtered_dict, x_axis, y_axis, minT, maxT, save_path=save_path, moving_avg_len=moving_avg_len, plot_fun=plot_fun)

plt.show()

In [ ]:
bh = 'corrected_coord_spin2_AhB'
y_axis1 = 'chiInertial_0'
y_axis2 = 'CoordSpinChiInertial_0'

X = data_dict[bh][x_axis]
Y1 = data_dict[bh][y_axis1]
Y2 = data_dict[bh][y_axis2]
plt.plot(X,Y1,label=y_axis1)
plt.plot(X,Y2,label=y_axis2)
plt.xlabel(x_axis)
# plt.ylabel(y_axis1+" - "+y_axis2)
plt.legend()
# plt.title()
plt.show()

In [ ]:
filtered_dict.keys()

In [ ]:
x_axis = 't'
y_axis = 'ChristodoulouMass'
minT = 500
maxT = 800
run1 = filtered_dict['AccTest_q1ns_Lev5_AhA']
# run1 = filtered_dict['AccTest_q1ns_Lev6p_AhA']
run2 = filtered_dict['AccTest_q1ns_Lev6p_AhA']
interp_grid_pts = run1[x_axis].size

In [ ]:
interp_run1 = CubicSpline(run1[x_axis].to_numpy(),run1[y_axis].to_numpy())
interp_run2 = CubicSpline(run2[x_axis].to_numpy(),run2[y_axis].to_numpy())
interp_grid = np.arange(minT,maxT,(maxT-minT)/interp_grid_pts)

plt.plot(interp_grid, interp_run2(interp_grid) - interp_run1(interp_grid))
plt.xlabel(x_axis)
plt.ylabel(y_axis)
plt.legend()

In [ ]:
plt.plot(interp_grid, interp_run2(interp_grid) - interp_run1(interp_grid))

In [ ]:
def inertial_dist(run_name:str, data_dict):
    ct = data_dict[f"{run_name}_AhA"].t
    dx = data_dict[f"{run_name}_AhA"].CoordCenterInertial_0 - data_dict[f"{run_name}_AhB"].CoordCenterInertial_0
    dy = data_dict[f"{run_name}_AhA"].CoordCenterInertial_1 - data_dict[f"{run_name}_AhB"].CoordCenterInertial_1
    dz = data_dict[f"{run_name}_AhA"].CoordCenterInertial_2 - data_dict[f"{run_name}_AhB"].CoordCenterInertial_2

    dx = np.sqrt(dx**2 + dy**2 + dz**2)

    return ct,dx


In [ ]:
for run_name in runs_to_plot.keys():
    ct,dx = inertial_dist(run_name,data_dict)
    plt.plot(ct,dx,label=run_name)
    plt.legend()

In [ ]:
print(data_dict.keys())
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(data_dict['76_ngd_master_mr1_50_3000_AhA'].describe())

# Combine all paraview files into a single file

In [ ]:
base_folder = Path("/central/groups/sxs/hchaudha/spec_runs/5_gd_SphKS_gauge_ID/Ev/")

file_pattern ="Lev2_A[A-Z]/Run/GaugeVis.pvd"
file_patternGrid ="Lev2_A[A-Z]/Run/GaugeVisGrid.pvd"
file_patternAll ="Lev2_A[A-Z]/Run/GuageVisAll.pvd"

combine_pvd_files(base_folder,file_pattern)
combine_pvd_files(base_folder,file_patternGrid)
combine_pvd_files(base_folder,file_patternAll)

```Python
# Create GaugeVis
command = f"cd {base_folder} && mkdir ./GaugeVis"
status = subprocess.run(command, capture_output=True, shell=True, text=True)
if status.returncode == 0:
  print(f"Succesfully created GaugeVis in {base_folder}")
else:
  sys.exit(
      f"GaugeVis creation failed in {base_folder} with error: \n {status.stderr}")

# Create GaugeVis subfolder
vtu_folder_path = base_folder+"/GaugeVis/GaugeVis"
command = f"mkdir {vtu_folder_path}"
status = subprocess.run(command, capture_output=True, shell=True, text=True)
if status.returncode == 0:
  print(f"Succesfully created {vtu_folder_path}")
else:
  sys.exit(
      f"GaugeVis creation failed as {vtu_folder_path} with error: \n {status.stderr}")


# Copy vtu files
GaugeVisFolder=[]

for paths in path_collection:
  GaugeVisFolder.append(paths[:-4])

for paths in GaugeVisFolder:
  command = f"cp {paths}/*.vtu {vtu_folder_path}/"
  status = subprocess.run(command, capture_output=True, shell=True, text=True)
  if status.returncode == 0:
    print(f"Succesfully copied vtu files from {paths}")
  else:
    sys.exit(
        f"Copying vtu files from {paths} failed with error: \n {status.stderr}")

```

# Profiler results

# Make report (do not run randomly)

In [ ]:
with open("./report_new_gauge.json") as report_data:
  data = json.load(report_data)

os.mkdir(data['report_folder'])

subfolders = []
for folders in data['runs_to_track']:
  subfolders_path = data['report_folder'] + "/" + path_to_folder_name(folders) + "/"
  print(subfolders_path)
  os.mkdir(subfolders_path)
  subfolders.append(subfolders_path)

In [ ]:
runs_still_going_on = True
while runs_still_going_on:
  # time.sleep(data['report_generation_frequency'])

  for i,run_folder_path in enumerate(data['runs_to_track']):
    # if is_the_current_run_going_on(run_folder_path) or True:
    if True:
      plots_for_a_folder(data['things_to_plot'],subfolders[i],run_folder_path)
    print(run_folder_path)


  runs_still_going_on = False
  print("all done")


### Save all columns and data files paths

In [ ]:
# Write all the cols in the dat files for reference
lev_golb="/panfs/ds09/sxs/himanshu/gauge_stuff/gauge_driver_runs/runs/gauge_driver_kerr_target_50_50_0_16_16_01/Ev/Lev1_AA"
dat_files_glob=lev_golb+"/Run/**/**.dat"
path_pattern = dat_files_glob

path_collection = []
for folder_name in glob.iglob(path_pattern, recursive=True):
    if os.path.isdir(folder_name) or os.path.isfile(folder_name):
        path_collection.append(folder_name)
        print(folder_name.split("/")[-1])


column_data_for_dat_files = {
  'columns_of_dat_files' : [
  ] 
}

for file_path in path_collection:
  file_name = file_path.split("/")[-1]
  columns_list =  list(read_dat_file(file_path).columns)
  column_data_for_dat_files['columns_of_dat_files'].append({
    'file_name': file_name,
    'file_path': file_path,
    'columns': columns_list
  })


with open('./column_data_for_dat_files.json', 'w') as outfile:
  json.dump(column_data_for_dat_files, outfile, indent=2)

In [ ]:
def JoinH5(h5_file_list, output_path, output_file_name):

  file_list_to_str = ""
  for h5file in h5_file_list:
    file_list_to_str += h5file + " "

  command = f"cd {output_path} && {spec_home}/Support/bin/JoinH5 -o {output_file_name} {file_list_to_str}"
  status = subprocess.run(command, capture_output=True, shell=True, text=True)
  if status.returncode == 0:
    print(f"Succesfully ran JoinH5 in {output_path}")
  else:
    sys.exit(
        f"JoinH5 failed in {output_path} with error: \n {status.stderr}")


def ExtractFromH5(h5_file, output_path):

  command = f"cd {output_path} && {spec_home}/Support/bin/ExtractFromH5 {h5_file}"
  status = subprocess.run(command, capture_output=True, shell=True, text=True)
  if status.returncode == 0:
    print(f"Succesfully ran ExtractFromH5 in {output_path}")
  else:
    sys.exit(
        f"ExtractFromH5 failed in {output_path} with error: \n {status.stderr}")



In [ ]:
output_base_path= "/panfs/ds09/sxs/himanshu/gauge_stuff/gauge_driver_runs/runs/profiler_results"


base_folder = "/panfs/ds09/sxs/himanshu/gauge_stuff/gauge_driver_runs/runs/49_ngd_weird_gauge_mr1"
file_pattern = base_folder+"/Ev/Lev1_A?/Run/Profiler.h5"

path_pattern = file_pattern
path_collection = []

# make a folder in the output directory
save_folder = output_base_path+"/"+base_folder.split("/")[-1]
os.mkdir(save_folder)


# Find all the files that match the required pattern of the file
for folder_name in glob.iglob(path_pattern, recursive=True):
    if os.path.isdir(folder_name) or os.path.isfile(folder_name):
        path_collection.append(folder_name)
        print(folder_name)

JoinH5(path_collection,save_folder,"Profiler.h5")
ExtractFromH5("Profiler.h5",save_folder)

# Save path of all the summary files in extracted data

file_pattern = base_folder+"/Ev/Lev1_A?/Run/Profiler.h5"

path_pattern = file_pattern
path_collection = []

# Find all the files that match the required pattern of the file
for folder_name in glob.iglob(path_pattern, recursive=True):
    if os.path.isdir(folder_name) or os.path.isfile(folder_name):
        path_collection.append(folder_name)
        print(folder_name)

In [ ]:
# Find all the Summary files 
summary_file_pattern = save_folder+"/**/Summary.txt"
summary_file_collection = []

for file_path in glob.iglob(summary_file_pattern, recursive=True):
    if os.path.isdir(file_path) or os.path.isfile(file_path):
        summary_file_collection.append(file_path)
        print(file_path)

summary_file_collection.sort()

In [ ]:
file_path = "/panfs/ds09/sxs/himanshu/gauge_stuff/gauge_driver_runs/runs/profiler_results/49_ngd_weird_gauge_mr1/extracted-Profiler/Step10522.dir/Summary.txt"



## AmrTolerances.input

In [ ]:
Lev=8
TruncationErrorMax = 0.000216536 * 4**(-Lev)
ProjectedConstraintsMax = 0.216536 * 4**(-Lev)
TruncationErrorMaxA = TruncationErrorMax*1.e-4
TruncationErrorMaxB = TruncationErrorMax*1.e-4

AhMaxRes  = TruncationErrorMax
AhMinRes  = AhMaxRes / 10.0

AhMaxTrunc=TruncationErrorMax
AhMinTrunc=AhMaxTrunc / 100.0

print(f"AhMinRes={AhMinRes};")
print(f"AhMaxRes={AhMaxRes};")
print(f"AhMinTrunc={AhMinTrunc};")
print(f"AhMaxTrunc={AhMaxTrunc};")
print(f"TruncationErrorMax={TruncationErrorMax};")
print(f"TruncationErrorMaxA={TruncationErrorMaxA};")
print(f"TruncationErrorMaxB={TruncationErrorMaxB};")
print(f"ProjectedConstraintsMax={ProjectedConstraintsMax};")

# Rough

In [ ]:
x = np.arange(100)/100
y1 = np.sin(x)
y2 = np.cos(x)

styles =  plt.style.available

for style in styles:
    print(style)
    plt.style.use(style)
    plt.plot(x,y1,label="y1asfasd")
    plt.plot(x,y2,label="y3asfasd")
    plt.title("asdf")
    plt.legend()
    plt.savefig(f"/panfs/ds09/sxs/himanshu/gauge_stuff/gauge_driver_runs/runs/make_report_del/{style}.png")
    plt.close()

In [ ]:
w = 4
print(np.convolve(np.arange(1,10),np.ones(w),'valid')/w)
print(np.arange(1,10))

In [ ]:
def moving_average(array,avg_len):
    return np.convolve(array,np.ones(avg_len))/avg_len

In [ ]:
runs_data_dict["AccTest_q1ns_Lev5"].iloc[1200:,1:].max()